# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y=loans_df['credit_ranking']
# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [9]:
# Define features set X by selecting all columns but credit_ranking
X=loans_df.drop(columns=['credit_ranking'])
# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [10]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# Fit the scaler to the features testing dataset
X_test_scaled = scaler.transform(X_test)
# Display the first three few entries of the scaled features
print("X_train_scaled sample:\n", X_train_scaled[:3])
print("X_test_scaled sample:\n", X_test_scaled[:3])

X_train_scaled sample:
 [[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]]
X_test_scaled sample:
 [[ 0.27988214 -0.67947695  1.87699031 -0.24897225  0.11388606 -0.64671182
  -0.11681233  0.96943574  1.43482818  0.00384497  0.05226031]
 [ 0.22360652  0.55286199  0.05722821  0.10845901  0.17694246  0.11885252
   0.72287966  0.31294474 -0.35064271 -0.17390392 -0.22508797]
 [ 1.18029216 -1.07158479  1.57369663  0.82332153  1.81640878 -0.93379844
  -0.92540463  0.13579637 -0.79701044  0.2408435   0.79185571]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
display ("Number of input features:",input_nodes)

'Number of input features:'

11

In [14]:
# Define the number of hidden nodes for the first hidden layer
hidden_units_layer1 = 64

# Define the input layer
input_layer = tf.keras.layers.Input(shape=(input_nodes,))

# Define the first hidden layer with ReLU activation
hidden_layer1 = tf.keras.layers.Dense(hidden_units_layer1, activation='relu')(input_layer)


In [15]:
# Define the number of hidden nodes for the first hidden layer
hidden_units_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_units_layer2 = 32

# Define the number of neurons in the output layer
output_neurons = 3 

In [16]:
# Create the Sequential model instance
mod = Sequential()

# Add the first hidden layer
mod.add(Dense(units=hidden_units_layer1, activation='relu', input_dim=input_nodes))

# Add the second hidden layer
mod.add(Dense(units=hidden_units_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
mod.add(Dense(units=output_neurons, activation='softmax'))


In [17]:
# Display the Sequential model summary
mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                768       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 2947 (11.51 KB)
Trainable params: 2947 (11.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [18]:
# Compile the Sequential model
mod.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Fit the model using 50 epochs and the training data
history = mod.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
32/32 [==============================] - 2s 14ms/step - loss: 0.8644 - accuracy: 0.5836 - val_loss: 0.6574 - val_accuracy: 0.7188
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6105 - accuracy: 0.7077 - val_loss: 0.5586 - val_accuracy: 0.7266
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5443 - accuracy: 0.7370 - val_loss: 0.5344 - val_accuracy: 0.7383
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5198 - accuracy: 0.7507 - val_loss: 0.5219 - val_accuracy: 0.7461
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5058 - accuracy: 0.7615 - val_loss: 0.5171 - val_accuracy: 0.7383
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4955 - accuracy: 0.7615 - val_loss: 0.5136 - val_accuracy: 0.7383
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 0.4866 - accuracy: 0.7634 - val_loss: 0.5100 - val_accuracy: 0.7422
Epoch 8/50
32/32 [=

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [21]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = mod.evaluate(X_test_scaled,y_test,verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.5551 - accuracy: 0.7375 - 34ms/epoch - 3ms/step
Loss: 0.5550984144210815, Accuracy: 0.737500011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [22]:
# Set the model's file path
model_path = 'student_loan_mod.keras'

# Export your model to a keras file
mod.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [24]:
# Set the model's file path
model_path = 'student_loan_mod.keras'

# Load the model to a new object
nn_import = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [25]:
# Make predictions with the test data
pred = mod.predict(X_test_scaled)

# Display a sample of the predictions
print('Sample predictions:')
print(pred[:5])

10/10 [==============================] - 0s 905us/step
Sample predictions:
[[7.52020895e-01 2.47966111e-01 1.30261924e-05]
 [5.62164426e-01 4.32189614e-01 5.64596662e-03]
 [1.36452690e-01 8.63308728e-01 2.38557957e-04]
 [1.66019395e-01 8.33768070e-01 2.12603321e-04]
 [1.04898931e-02 9.89510059e-01 1.06868185e-07]]


In [28]:
# Save the predictions to a DataFrame and round the predictions to binary results
pred_df = pd.DataFrame(pred, columns=['Class_0', 'Class_1', 'Class_2'])
pred_df['Predicted_Class'] = pred_df.idxmax(axis=1)
pred_df['Actual'] = y_test
pred_df['Predicted_Binary'] = (pred_df['Predicted_Class'] == 'Class_2').astype(int)

# Display a sample of the predictions
print(pred_df.head())

    Class_0   Class_1       Class_2 Predicted_Class  Actual  Predicted_Binary
0  0.752021  0.247966  1.302619e-05         Class_0     NaN                 0
1  0.562164  0.432190  5.645967e-03         Class_0     NaN                 0
2  0.136453  0.863309  2.385580e-04         Class_1     NaN                 0
3  0.166019  0.833768  2.126033e-04         Class_1     1.0                 0
4  0.010490  0.989510  1.068682e-07         Class_1     NaN                 0


### Step 4: Display a classification report with the y test data and predictions

In [29]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, pred_df['Predicted_Binary']))

              precision    recall  f1-score   support

           0       0.48      1.00      0.65       154
           1       0.00      0.00      0.00       166

    accuracy                           0.48       320
   macro avg       0.24      0.50      0.32       320
weighted avg       0.23      0.48      0.31       320



c:\Users\revcr\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\revcr\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\revcr\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

